<a href="https://colab.research.google.com/github/garrisonreid/DurhamFoodLocations/blob/master/Durham_Summer_Meals_Map_2019_Public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pandas
!pip install folium
!pip install geopandas

     |████████████████████████████████| 901kB 5.0MB/s 
     |████████████████████████████████| 11.2MB 39.5MB/s 
     |████████████████████████████████| 17.9MB 34.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/db/bf/bc/06a3e1bfe0ab27d2e720ceb3cff3159398d92644c0cec2c125
Successfully built munch


In [0]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium

df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vQa7xqdHPBv0l001dydV5JuDvAaUh7um07FkH3YVntOj-v43BxUcZnK3L0XgqFUXncfSA38IM77hU14/pub?gid=794529442&single=true&output=csv")
summerLocations = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vSXOeU0JKQgqTXpiwnvcO3cXe1iAIGVYwxzKKB5rPgT1sFUTVQcsjtfwzH_JUMpmKL7KPNg-o_bG5y0/pub?gid=2046019608&single=true&output=csv")
df2 = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vRELikdBBJHlgSjMkGjNP6CQL5VwxrD92vVIdMDAfi58x3VHJG0dY7SjhvhGOdt-0b42b08j-YlDBY8/pub?output=csv")
# summerLocations[summerLocations['New Sites'] == "Returning Site"].head()

In [118]:
import matplotlib.pyplot as plt
!pip install descartes
!pip install plotly
# !pip install geoplot
# import geoplot.crs as gcrs

In [0]:
def make_point(row):
    return Point(row.lng, row.lat)

def make_marker(row):
  return Markers(row.Label)
  
pointsRL = summerLocations.apply(make_point, axis=1)
pickupLocationsReturning = gpd.GeoDataFrame(summerLocations[summerLocations['New Sites'] == "Returning Site"], geometry=pointsRL)
pickupLocationsReturning.crs = {'init': 'epsg:4326'}
# pickupLocationsReturning.head()

pointsNL = summerLocations.apply(make_point, axis=1)
pickupLocationsNew = gpd.GeoDataFrame(summerLocations[summerLocations['New Sites'] == "New Site"], geometry=pointsNL)
pickupLocationsNew.crs = {'init': 'epsg:4326'}
# pickupLocations.head()


# points = df.apply(make_point, axis=1)
# pickupLocations = gpd.GeoDataFrame(df, geometry=points)
# pickupLocations.crs = {'init': 'epsg:4326'}
# # pickupLocations.head()



pointsC = df2.apply(make_point, axis=1)
churchLocations = gpd.GeoDataFrame(df2, geometry=pointsC)
churchLocations.crs = {'init': 'epsg:4326'}
# churchLocations.head()

In [0]:
# pickupLocationsReturning.plot()

In [0]:
# churchLocations.plot()

In [128]:
dcb = gpd.read_file("https://opendata.arcgis.com/datasets/01b3cb54b5a64105a742d5c5b4217599_0.geojson")

bounds = folium.GeoJson(dcb).get_bounds()
center = [(bounds[0][0] + bounds[1][0]) / 2, (bounds[0][1] + bounds[1][1]) / 2]
m = folium.Map(
  location=center,
  zoom_start=10.75,
   tiles=None
)

folium.TileLayer (
  tiles='openstreetmap',
  show=True,
  control=False
).add_to(m)

# # Add county boundary
# folium.Choropleth(
#     geo_data=dcb['geometry'],
#     name='County Boundary',
#     fill_color='#00000000',
#     fill_opacity=0.0,
#     line_opacity=0.8,
#     line_color='gray',
#     line_weight=4
# ).add_to(m)

In [129]:
tracts = gpd.read_file("https://raw.githubusercontent.com/garrisonreid/DurhamFoodLocations/master/durham_tracts.geojson")
tracts.head()

enrollment = pd.DataFrame(tracts)
enrollment.head()

,GEOID,GSPL,Label,geometry
0,37063001002,612,"Census Tract 10.02, Durham County, North Carolina","(POLYGON ((-78.88070399999999 35.998738, -78.8..."
1,37063001100,412,"Census Tract 11, Durham County, North Carolina","(POLYGON ((-78.89711699999999 35.990498, -78.8..."
2,37063001301,59,"Census Tract 13.01, Durham County, North Carolina","(POLYGON ((-78.90840799999999 35.97956, -78.90..."
3,37063001303,81,"Census Tract 13.03, Durham County, North Carolina","(POLYGON ((-78.908708 35.975495, -78.908700999..."
4,37063001304,536,"Census Tract 13.04, Durham County, North Carolina","(POLYGON ((-78.92167600000001 35.967415, -78.9..."


In [0]:
# # Add census tracts
# folium.Choropleth(
#     geo_data=tracts,
#     data=enrollment,
#     columns=['GEOID','GSPL'],
#     name="Census Tracts",
# #     key_on='feature.properties.GEOID',
#     fill_color='YlOrRd',
#     fill_opacity=0.0,
#     line_color='black',
#     line_opacity=0.5,
# #     popup=enrollment['Label'],
#     control=False,
# #     legend_name='Grade school enrollment below the poverty line',
#     weight=1
# ).add_to(m)

In [131]:
churchmarkers = folium.FeatureGroup(name='Churches', control=True)
pickuplocationsR = folium.FeatureGroup(name='Returning Pickup Locations', control=True)
pickuplocationsN = folium.FeatureGroup(name='New Pickup Locations', control=True)


# pickuplocations = folium.FeatureGroup(name='Test', control=True)


# # Red markers are churches
# for index, row in churchLocations.iterrows():
#   folium.CircleMarker([row['lat'], row['lng']], radius=1.5, popup=row['Label'], layer='Churches', color='red', fill_color='red').add_to(churchmarkers)

# # Blue markers are 2018 pickup locations
for index, row in pickupLocationsReturning.iterrows():
  folium.CircleMarker([row['lat'], row['lng']], radius=1.5, layer='Returning Pickup Locations',color='#1324bc', fill_color='#1324bc', popup = "<strong>" + row['Site Name']  + "</strong><br/>" + row['Address']).add_to(pickuplocationsR)

# # Green markers are 2019 pickup locations
for index, row in pickupLocationsNew.iterrows():
  folium.CircleMarker([row['lat'], row['lng']], radius=1.5, layer='New Pickup Locations',color='#00cc0a', fill_color='#00cc0a', width=500, popup = "<strong>" + row['Site Name']  + "</strong><br/>" + row['Address']).add_to(pickuplocationsN)
# # # Blue markers are 2018 pickup locations
# for index, row in pickupLocations.iterrows():
#   folium.CircleMarker([row['lat'], row['lng']], radius=1.5, popup=row['Label'], layer='2018 Pickup Locations',color='blue', fill_color='blue').add_to(pickuplocations)  
  
  
# churchmarkers.add_to(m)
pickuplocationsR.add_to(m)
pickuplocationsN.add_to(m)
# pickuplocations.add_to(m)

In [132]:
edci = gpd.read_file("https://raw.githubusercontent.com/garrisonreid/DurhamFoodLocations/master/EDCI_Zone.geojson")
edci.head()

# # # Add EDCI Zone boundary
# folium.Choropleth(
#     geo_data=edci['geometry'],
#     name='EDCI',
#     fill_color='yellow',
#     fill_opacity=0.35,
#     line_color='orange',
#     line_opacity=1.0,
#     line_weight=4,
# #     show=True,
# ).add_to(m)

folium.LayerControl(collapsed=False).add_to(m)

In [133]:
m